---
title: "Replication of Gianattasio et al. (2019)"
author: "Ralph Møller Trane"
format: 
  gfm:
    output-file: README.md
bibliography: references.bib
editor_options: 
  chunk_output_type: console
engine: jupyter
---


## Introduction

This repo aims at replicating the results of @gianattasioComparisonMethodsAlgorithmic2019. The greater idea behind this is a desire to implement the algorithms in R for future research purposes, specifically validating the main findings from @gianattasioComparisonMethodsAlgorithmic2019 by way of analyzing WLS data. To that end, we want to ensure that our implementation of the algorithms are aligned with previous implementations. Therefore, we create this pipeline with instructions to recreate the training and validation datasets from @gianattasioComparisonMethodsAlgorithmic2019 that can serve to ensure the correctness of our implementations of algorithms.

## Requirments

This repo was developed for use on a linux machine with a working installation of R and SAS available. We provide a `Makefile` to easily recreate the training and validation datasets. The main pieces of software used are 

* `SAS` version 9.4 with analytical products
    * `SAS/STAT 15.3`
    * `SAS/ETS 15.3`
    * `SAS/OR 15.3`
    * `SAS/IML 15.3`
    * `SAS/QC 15.3`
* `R 4.4.0` with the following packages installed:
    * `haven` version `2.5.4`
    * `readr` version `2.1.5` 
    * `tidyr` version `1.3.1` 
    * Things will most likely also work with previous versions
* `Make` version `4.2.1`

## Step-by-step guide

We will go through the creation of the validation and test datasets step-by-step below. However, to simply quickly and easily create the data files, you can clone this repo to your local machine, download the files mentioned below to the folder `data/HRS-zips`, then run `make all` from the root directory of this repo. Note that the zip-files with the ADAMS data are locked with a password. If running the code below locally, replace `MYPASSWORD` with the password provided by HRS when you requested access to the ADAMS data.

```
git clone https://github.com/rmtrane/replication-of-gianattasio.git ./

cd replication-of-gianattasio/

mkdir data/HRS-zips # put downloaded .zip files here

make all password=MYPASSWORD
```

### Prepare the data from HRS

#### Download appropriate files

The following .zip files should be downloaded from the [HRS Data Downloads site](https://hrsdata.isr.umich.edu/data-products/) and placed in the `data/HRS-zips` folder. We include links to data pages with details for each set of files, and also direct links for download (file names formatted as `file.zip` will take you straight to download the given file). A login is needed to download the data.

Note: make sure your browser does not automatically open files when downloaded. If .zip files are automatically opened, they will be unzipped, and not fit the patterns expected. (On Safari on macOS: got Settings -> General -> uncheck 'Open "safe" files after downloading'.)

* HRS Survey Data:
  - [1998 HRS Core](https://hrsdata.isr.umich.edu/data-products/1998-hrs-core): [`h98da.zip`](https://hrsdata.isr.umich.edu/data-file-download/5515) and [`h98sas.zip`](https://hrsdata.isr.umich.edu/data-file-download/5625)
  - [2000 HRS Core](https://hrsdata.isr.umich.edu/data-products/2000-hrs-core): [`h00da.zip`](https://hrsdata.isr.umich.edu/data-file-download/5508) and [`h00sas.zip`](https://hrsdata.isr.umich.edu/data-file-download/5640)
  - [2002 HRS Core](https://hrsdata.isr.umich.edu/data-products/2002-hrs-core): [`h02da.zip`](https://hrsdata.isr.umich.edu/data-file-download/5506) and [`h02sas.zip`](https://hrsdata.isr.umich.edu/data-file-download/5637)
  - [2004 HRS Core](https://hrsdata.isr.umich.edu/data-products/2004-hrs-core): [`h04da.zip`](https://hrsdata.isr.umich.edu/data-file-download/5516) and [`h04sas.zip`](https://hrsdata.isr.umich.edu/data-file-download/5645)
  - [2006 HRS Core](https://hrsdata.isr.umich.edu/data-products/2006-hrs-core): [`h06da.zip`](https://hrsdata.isr.umich.edu/data-file-download/9480) and [`h06sas.zip`](https://hrsdata.isr.umich.edu/data-file-download/9483)
  - [2008 HRS Core](https://hrsdata.isr.umich.edu/data-products/2008-hrs-core): [`h08da.zip`](https://hrsdata.isr.umich.edu/data-file-download/5510) and [`h08sas.zip`](https://hrsdata.isr.umich.edu/data-file-download/5652)
  - [2010 HRS Core](https://hrsdata.isr.umich.edu/data-products/2010-hrs-core): [`h10da.zip`](https://hrsdata.isr.umich.edu/data-file-download/9464) and [`h10sas.zip`](https://hrsdata.isr.umich.edu/data-file-download/9466)

* RAND HRS Data:
  - We need an older version of the RAND data, the so-called Version P. This can be found under [archived data versions](https://hrsdata.isr.umich.edu/data-products/rand-hrs-archived-data-products).<br>
    The file needed: 
        - RAND HRS Version P [`randhrsp_archive_SAS.zip`](https://hrsdata.isr.umich.edu/data-file-download/5414)

* ADAMS Data:
  - From the main HRS Data Downloads page, navigate to [HRS Sensitive Health Data](https://hrsdata.isr.umich.edu/data-products/sensitive-health). Note: additional permissions are needed to access this part of the data.<br>
    The files to be downloaded are:
      - [Wave A](https://hrsdata.isr.umich.edu/data-products/aging-demographics-and-memory-study-adams-wave): [adams1a.zip](https://hrsdata.isr.umich.edu/data-file-download/5539)
      - [Wave B](https://hrsdata.isr.umich.edu/data-products/aging-demographics-and-memory-study-adams-wave-b): [adams1b.zip](https://hrsdata.isr.umich.edu/data-file-download/5616)
      - [CrossWave](https://hrsdata.isr.umich.edu/data-products/aging-demographics-and-memory-study-adams-cross-wave-tracker-file): [adams1trk.zip](https://hrsdata.isr.umich.edu/data-file-download/5570) 
      
* Hurd Probabilities:
  - These are provided as a "Contributed Project" on the HRS website; follow [this direct link](https://hrsdata.isr.umich.edu/data-products/dementia-predicted-probabilities-files) to the page where the following zip file can be downloaded:
    - [DementiaPredictedProbabilities.zip](https://hrsdata.isr.umich.edu/data-file-download/5578)

#### Unzip all files (`make unzip_all password=MYPASSWORD`)

All the `hXXXX.zip` files are in pairs: `hXXda.zip` and `hXXsas.zip`. Unzip each pair, and move the resulting folders to `data/HRS-unzips/hXX`. 

For the `adams1X.zip` files, unzip them and move the resulting folders to `data/HRS-unzips`. In each of the folders, you will find additional `.zip` files. Unzip the two called `adams1Xda.zip` and `adams1Xsas.zip`. Rename them to `da.zip` and `sas.zip`, respectively. 

For the `DementiaPredictedProbabilities.zip` and `randhrs_p_archive_SAS.zip` files, unzip them and move to `data/HRS-unzips/hurd` and `data/HRS-unzips/rand`, respectively. 

When completed, you should have a folder structure similar to what is illustrated below. I've excluded the files in the `da` and `sas` subfolders since there are a lot... We'll get back to these soon.

```
data/HRS-unzips
├── a95
│   ├── da
│   └── sas
├── adams1a
│   ├── da
│   └── sas
├── adams1b
│   ├── da
│   └── sas
├── adams1trk
│   ├── da
│   └── sas
├── h00
│   ├── da
│   └── sas
├── h02
│   ├── da
│   └── sas
├── h04
│   ├── da
│   └── sas
├── h06
│   ├── da
│   └── sas
├── h08
│   ├── da
│   └── sas
├── h10
│   ├── da
│   └── sas
├── h96
│   ├── da
│   └── sas
└── h98
│   ├── da
│   └── sas
├── hurd
│   ├── DescriptionOfPredictedProbabilities.pdf
│   ├── pdem_withvarnames.dta
│   └── pdem_withvarnames.sas7bdat
└── rand
    ├── ArchiveREADME.pdf
    ├── incwlth_p.sas7bdat
    ├── randhrs_P.pdf
    ├── randiwp.pdf
    ├── rndhrs_p.sas7bdat
    └── sasfmts.sas7bdat
```

#### Update SAS scripts (`make .update_all_sas`)

Ultimately, we need to run a bunch of SAS scripts that take `.da` files and create `.sas7bdat` files for later use. All the SAS files we need to run are in the `HRS-unzips/*/sas/` folders. Before we can run them, we need to update the some paths. 

Take, as an example, the file `data/HRS-unzips/h98/sas/h98a_r.sas`. This file is about 300 lines long, but the essence of the file is included below. It essentially extracts the data needed from the data file `data/HRS-unzips/h98/da/h98a_r.da`, and saves it to a new data set. 

```
libname EXTRACT 'c:\hrs1998\sas\' ; 

DATA EXTRACT.H98A_R;
INFILE 'c:\hrs1998\data\H98A_R.DA'  LRECL=192; 

INPUT 
  ... specify input columns...
;


LABEL
  ... variable labels...
;

run;

DATA EXTRACT.H98A_R;
SET  EXTRACT.H98A_R;
FORMAT 
  ... formatting of variables...
;
run;

```

To be able to run this SAS file, we need to fix the file paths. Set the `libname EXTRACT` to the folder `data/SAS/HRS/`, but include the full path. Also, fix the path to the input data file. In the end, the beginning of the file should look like this:

```
libname EXTRACT '/path/to/root/folder/data/SAS/HRS/';

DATA EXTRACT.H98A_R;
INFILE '/path/to/root/folder/data/HRS-unzips/h98/da/h98a_r.da' LRECL=192;
```

The rest of the file remains unchanged. Save this file to a new folder `data/HRS-unzips/h98/new_sas`. Repeat with all SAS files in the `data/HRS-unzips/h*` folders.

For the SAS files in `data/HRS-unzips/adams1*/sas`, change the libname paths to `/path/to/root/folder/data/SAS/ADAMS`.

#### Run SAS scripts (`make .run_all_sas`)

Once we have created updated SAS files, we need to run them all. Before doing so, make sure you've created the folders `data/SAS/HRS` and `data/SAS/ADAMS`. 

Once you've run all SAS files, you will have a total of 304 `.sas7bdat` files in `data/SAS/HRS` and 29 `.sas7bdat` files in `data/SAS/ADAMS`.

#### Prepare RAND data (`make data/SAS/rand/formats.sas7bcat`)

To prepare the RAND data, we copy `rndhrs_p.sas7bdat` and `sasfmts.sas7bdat` from `data/HRS-unzips/rand` to `data/SAS/rand`. We then create `data/SAS/rand/formats.sas7bcat` using the following SAS scripts:

```
libname library '/path/to/root/folder/data/SAS/rand';
  proc format library=library cntlin=library.sasfmts;
run;
```

#### Prepare HURD data (`make data/SAS/created/hurdprobabilities_wide.csv`)

The probabilities included from `data/HRS-unzips/hurd/pdem_withvarnames.sas7bdat` need to be in a wide format for the SAS files from [`powerepilab/AD_algorithm_comparison`](https://github.com/powerepilab/AD_algorithm_comparison) to be able to use them. The R-script can be run from the root folder to create an appropriate .csv file.


```{r}
library(tidyr)

haven::read_sas('data/HRS-unzips/hurd/pdem_withvarnames.sas7bdat') %>% 
  pivot_wider(
    names_from = prediction_year, 
    values_from = prob_dementia, 
    names_prefix = 'hurd_prob_'
  ) %>% 
  readr::write_csv('data/SAS/created/hurdprobabilities_wide.csv', na = '.')
```


Note: the files from [`powerepilab/AD_algorithm_comparison`](https://github.com/powerepilab/AD_algorithm_comparison) rely on a dataset similar to the .csv file created here, but since my SAS skills a lacking a bit, I wasn't able to create this in SAS. Instead, we will modify the SAS scripts later to read in the probabilities from this .csv file.

### Prepare SAS scripts to create training and validation data

We now have all the data ready to create the training and validation data sets used in @gianattasioComparisonMethodsAlgorithmic2019. Now, we will prepare the SAS scripts that will eventually create the two data sets.

#### Download from [`powerepilab/AD_algorithm_comparison`](https://github.com/powerepilab/AD_algorithm_comparison) (`make AD_algorithm_comparison/touch`)

Clone the github repo into a new folder in your root directory. Call this folder `AD_algorithm_comparison`. To make sure no new changes have been pushed to the repo, you can check out the commit I used when creating this. To do so, run the following command from inside the `AD_algorithm_comparison` folder.

```
git reset --hard 1338e71
```

You should now see the following files in the folder `AD_algorithm_comparison`

```
AD_algorithm_comparison/
├── 1a. Extract self-response variables from RANDp _ 2018.01.17.sas
├── 1b. Extract proxy variables from core HRS _ 2018.01.17.sas
├── 2. Create lags,leads, merge with ADAMS, set up regression vars _ 2018.01.17.sas
├── 3. Assign algorithmic dementia diagnoses and create HRSt HRSv datasets_ 2018.03.02.sas
├── Construct dataset of existing algorithm classifications for waves 2000-2014_2020_0110.sas
└── README.md
```

#### Adjust SAS scripts (`make updated_AD_algorithm_comparison/touch`)

As with the previous SAS scripts, we also need to adjust the paths in the SAS scripts just downloaded. 

For the file `AD_algorithm_comparison/1a. Extract self-response variables from RANDp _ 2018.01.17.sas`, change the beginning from 

```
libname adams 'F:\power\HRS\ADAMS Wave A';
libname atrk 'F:\power\HRS\ADAMS CrossWave';
libname x 'F:\power\HRS\DerivedData\AD_Disparities_AlgorithmDev\Data 2018_0105'; /*derived hrs files*/
libname hrs 'F:\power\HRS\HRS data (raw)\SAS datasets'; /*raw hrs files, including Hurd probabilities*/
libname rand 'F:\power\HRS\RAND_HRS\sasdata';
```

to

```
libname adams 'path/to/root/folder/data/SAS/ADAMS';
libname atrk 'path/to/root/folder/data/SAS/ADAMS';
libname x 'path/to/root/folder/data/SAS/created';
libname hrs 'path/to/root/folder/data/SAS/HRS';
libname rand 'path/to/root/folder/data/SAS/rand';
```

and save the updated file as `updated_AD_algorithm_comparison/1a_extract_self_response_variables.sas`.

For `AD_algorithm_comparison/1b.  Extract proxy variables from core HRS _ 2018.01.17.sas`, change the line 
```
%include "F:\power\HRS\Projects\Ad_Disparities_AlgorithmDev\SAS Programs\Code_2018_0117\1a. Extract self-response variables from RANDp _ 2018.01.17.sas";
```
to
```
%include "/path/to/root/folder/updated_AD_algorithm_comparison/1a_extact_self_response_variables.sas";
```
We also need to make sure we keep the Vice President variable that is (later used for Table 1)[#checks]

### Create training and validation data

## Checks

To make sure everything worked as intended, we recreate Table 1 of @gianattasioComparisonMethodsAlgorithmic2019. 

<details>

<summary>R code for recreating Table 1</summary>

First, the used libraries and reading in the data that was created from the SAS.


```{r}
library(tidyverse)
library(gt)
library(gtsummary)

hrs_training <- haven::read_sas(here::here("data/SAS/created/hrst_2018_0302.sas7bdat"))
hrs_validation <- haven::read_sas(here::here("data/SAS/created/hrsv_2018_0302.sas7bdat"))

master_ad <- haven::read_sas(here::here("data/SAS/created/master_ad_2018_0117.sas7bdat"))
```


Lots of the predictors are not present in the `hrs_training` and `hrs_validation` data, but rather included in the `master_ad` data. So, we add the predictors to the data sets. 

For the training data, this is straightforward, since all training data is from the ADAMS Wave A data set.


```{r}
hrs_training <- left_join(
    hrs_training,
    master_ad %>% 
      select(
        HHID, 
        PN, 
        iadl = iadl5_A,
        adl = adl5_A,
        iqcode = iqcode_A,
        jorm_symptoms = jorm5symp_A,
        pr_memsc = pr_memsc1_A,
        iwercog = iwercog_A,
        immediate_word_recall = iword_A,
        delayed_word_recall = dword_A,
        serial7 = serial7_A,
        dates = dates_A,
        cactus = cact_A,
        scissors = scis_A,
        president = pres_A,
        vp = vp_A,
        count = ticscount1or2_A
      )
  ) %>% 
    mutate(dataset = 'Training')
```


For the validation data, it is a bit more complicated. Here, we need to pull the correct ADAMS wave for each row of the validation data. 


```{r}
hrs_validation <- hrs_validation %>% 
  left_join(
    master_ad %>% 
      select(
        HHID, 
        PN, 
        matches("^iadl5_[BCD]$"),
        matches("^adl5_[BCD]$"),
        matches("^iqcode_[BCD]$"),
        matches("^jorm5symp_[BCD]$"),
        matches("^pr_memsc1_[BCD]$"),
        matches("^iwercog_[BCD]$"),
        matches("^iword_[BCD]$"),
        matches("^dword_[BCD]$"),
        matches("^serial7_[BCD]$"),
        matches("^dates_[BCD]$"),
        matches("^cact_[BCD]$"),
        matches("^scis_[BCD]$"),
        matches("^pres_[BCD]$"),
        matches("^vp_[BCD]$"),
        matches("^ticscount1or2_[BCD]$")
      ) %>% 
      rename_with(
        .fn = \(x) str_remove(x, '_'),
        .cols = matches("^pr_memsc1_[BCD]$")
      ) %>% 
      pivot_longer(
        cols = matches("_[BCD]$"),
        names_to = c(".value", "ADAMSwave"),
        names_sep = "_",
      ) %>% 
      rename(
        iadl = iadl5,
        adl = adl5,
        jorm_symptoms = jorm5symp,
        pr_memsc = prmemsc1,
        immediate_word_recall = iword,
        delayed_word_recall = dword,
        cactus = cact,
        scissors = scis,
        president = pres,
        count = ticscount1or2
      ) %>% 
      mutate(
        ADAMSwave = str_to_lower(ADAMSwave)
      )
  ) %>% 
  mutate(
    dataset = 'Validation'
  )
```


Finally, we recreate Table 1.


```{r}
## We use sjmisc::add_rows to preserve column labels. (dplyr::bind_rows removes labels.)
table_1 <- sjmisc::add_rows(
  hrs_training,
  hrs_validation
) %>% 
  select(
    dement,
    hrs_age,
    proxy,
    female,
    edu_hurd,
    raceeth4,
    immediate_word_recall,
    delayed_word_recall,
    serial7,
    dates,
    cactus,
    scissors,
    president,
    vp,
    count,
    iwercog,
    pr_memsc,
    iqcode,
    jorm_symptoms,
    adl, 
    iadl,
    dataset
  ) %>% 
  mutate(
    ## Replace self-response for proxy that are 0 with NA
    immediate_word_recall = if_else(proxy == 0, immediate_word_recall, NA),
    delayed_word_recall = if_else(proxy == 0, delayed_word_recall, NA),
  ) %>% 
  gtsummary::tbl_summary(
    by = 'dataset',
    statistic = list(all_continuous() ~ "{mean} ({sd})", all_categorical() ~ "{n} ({p}%)"),
    type = list(
      adl = 'continuous',
      iadl = "continuous",
      jorm_symptoms = "continuous",
      pr_memsc = "continuous",
      serial7 = 'continuous',
      dates = 'continuous'
    ),
    digits = list(
      all_continuous() ~ 1
    )
  ) %>% 
  modify_header(
    label ~ "**Outcomes and Predictors**"
  ) %>% 
  modify_footnote(
    all_stat_cols() ~ NA
  ) %>% 
  modify_spanning_header(
    c(stat_1, stat_2) ~ "**Mean (SD) or N (%)**"
  ) %>% 
  modify_table_body(
    mutate,
    groupname_col = case_match(
      variable,
      'dement' ~ 'Dementia Outcomes',
      c("hrs_age", "proxy", "female", "edu_hurd", "raceeth4") ~ 'Demographics',
      c('immediate_word_recall', 'delayed_word_recall', 'serial7', 
        'dates', 'cactus', 'scissors', 'president', 'vp', 'count') ~ 'Cognition (self-response)',
      c("iwercog", "pr_memsc", "iqcode", "jorm_symptoms") ~ 'Cognition (proxy)',
      c('adl', 'iadl') ~ 'Physical functioning limitations'
    )
  ) %>% 
  modify_column_indent(
    columns = label,
    rows = row_type %in% 'label',
    indent = 4L
  ) %>% 
  modify_column_indent(
    columns = label,
    rows = !row_type %in% 'label',
    indent = 8L
  ) %>% 
  as_gt() %>%
  cols_width(
    label ~ px(550)
  ) %>% 
  tab_options(
    row_group.font.weight = "500",
    row_group.background.color = 'white'
  ) 
```


</details>

A few things to note about the table below:

* Some rows indicate that there are a number of "unknown" (or missing) values. For the variables in the "Cognition (self-response)" group, those who were evaluated by proxy are excluded. Therefore, unknown = number of proxy cases. For "Cognition (proxy)", unknown = number of NOT proxy cases. 
* There is a mistake in the label for the `delayed word recall` variable: this is in fact on a scale from 0 to 10, not 0 to 1. 
* The proxy rated memory score is for some reason on a scale of 0 to 4, whereas in Table 1 of @gianattasioComparisonMethodsAlgorithmic2019 it is on a scale from 1 to 5. Simply add 1 to the means, and we see that they match.

<style>
tr.even {background-color: white;}
</style>


```{r}
#| output: asis
table_1
```